Crm Token

In [1]:
import pandas as pd
import json
import requests
import time
import threading
import pyodbc
from concurrent.futures import ThreadPoolExecutor
import ast
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import os
from datetime import datetime, date

'''
ask token
'''    
url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
payload = {
    "grant_type": "password",
    "client_id": "351fb11916fc85504ba5f6ec3b96988b",
    "client_secret": "8f9355878504005ba782ca049d013d4e",
    "redirect_uri": "https://crm-p10.xiaoshouyi.com",
    "username": "11020964@twkd.com",
    "password": "Kd0627ubGHagpQ"
}

response = requests.post(url, data=payload)
content = response.json()
ac_token = content["access_token"]

有效客戶

In [2]:

date_0 = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
date_0_scrm = int(date_0.timestamp() * 1000)

'''
select CompanyContact data
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
comp_cont = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
            SELECT id,
                   contactCode__c__c AS contact_code,
                   customItem2__c.contactName AS displayname,
                   name AS ContactID,
                   contactPhone__c__c AS MobilePhone,
                   customItem5__c AS CompanyTypeID,
                   customItem1__c AS customItem8__c,
                   customItem8__c AS CompanyID,
                   dimDepart,
                   ownerId,
                   customItem1__c.accountName AS CoFullName,
                   customItem9__c AS CoShortName,
                   customItem24__c AS employed,
                   customItem109__c AS 勿擾,
                   customItem37__c AS DataRegion,
                   customItem38__c AS 拜電訪標籤,
                   customItem106__c AS LastWorkDescription,
                   customItem103__c AS LastWorkDate,
                   customItem42__c AS Invalid,
                   customItem83__c AS 無效電訪,
                   customItem95__c AS JobTypeID,
                   customItem1__c.SAP_CompanyID__c AS SAP_CompanyID,
                   customItem1__c.Phone__c AS Company_Phone,
                   customItem1__c.customItem227__c AS CompanyID_old,
                   customItem1__c.customItem278__c AS Inactive,
                   customItem50__c as 空號,
                   customItem51__c as 停機,
                   customItem1__c.region__c.customItem9__c as 城市,
                   customItem30__c as 公司地址
            FROM customEntity22__c
            WHERE customItem37__c NOT LIKE 'TW%' AND customItem37__c NOT LIKE 'KDED%'
            AND (customItem5__c LIKE '%C%'
            OR customItem5__c LIKE '%D%')
            ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    comp_cont = pd.concat([comp_cont, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
comp_cont = comp_cont[~comp_cont['CoFullName'].str.contains('Dup>')]
print(comp_cont['DataRegion'].unique())
 
# JobtypeID
comp_cont['JobTypeID'] = comp_cont['JobTypeID'].str.get(0).str[:3]
comp_cont['JobTypeID'] = comp_cont['JobTypeID'].fillna('')
comp_cont = comp_cont[comp_cont['JobTypeID'].isin(["001","002","003","004","005","006","007","009", "010","011","015","017", ""])]

# Contact Mobile
comp_cont['MobilePhone'] = comp_cont['MobilePhone'].str.replace(r'\D', '', regex=True)
comp_cont.loc[comp_cont['MobilePhone'].str.contains("000000"), 'MobilePhone'] = ""
comp_cont.loc[comp_cont['MobilePhone'].str.len() < 6, 'MobilePhone'] = ""

# Company Phone
comp_cont['Company_Phone'] = comp_cont['Company_Phone'].str.replace(r'\D', '', regex=True)
comp_cont.loc[comp_cont['Company_Phone'].str.len() < 7, 'Company_Phone'] = ""

# Co Name
wordlist = "Closed|搬遷|倒閉|歇業|停業|轉行|退休|過世|廢止|解散|燈箱|群組|支援|留守|教育訓練|無效拜訪|資料不全"
comp_cont = comp_cont[~comp_cont['CoFullName'].str.contains(wordlist)]
comp_cont = comp_cont[~comp_cont['CoShortName'].str.contains(wordlist)]

# employed
comp_cont.sort_values(by='id', ascending=False, inplace=True)
comp_cont.drop_duplicates(subset='contact_code', keep='first', inplace=True)
comp_cont['employed'] = comp_cont['employed'].str.get(0)
comp_cont.loc[comp_cont['employed'].str.contains("離職|离职", regex=True, na=False), 'Company_Phone'] = ""

# Invalid
comp_cont['Invalid'] = comp_cont['Invalid'].str.get(0)
comp_cont['Inactive'] = comp_cont['Inactive'].str.get(0)
comp_cont['Invalid'] = comp_cont['Invalid'].fillna("否")
comp_cont['停機'] = comp_cont['停機'].fillna("0")
comp_cont['空號'] = comp_cont['空號'].fillna("0")
comp_cont['Inactive'] = comp_cont['Inactive'].fillna("否")
comp_cont.loc[(comp_cont['Invalid'] == "是") | (comp_cont['停機'] == "1") | (comp_cont['空號'] == "1"), 'MobilePhone'] = ""
comp_cont.loc[comp_cont['Inactive'] == "是", 'Company_Phone'] = ""

# SAP credit
# comp_cont.loc[comp_cont['SAP_CompanyID'].isin(credit_data[credit_data['SAP信用管制'] == "Restrain"]['SAP公司代號']), 'Company_Phone'] = ""

# 勿擾
comp_cont['勿擾'] = comp_cont['勿擾'].fillna('')
comp_cont['勿擾'] = comp_cont['勿擾'].astype(str)
comp_cont = comp_cont[~comp_cont['勿擾'].str.contains("勿電訪|勿傳送簡訊")]

# Remove blank mobile/phone and remove duplicate
comp_cont = comp_cont[~((comp_cont['MobilePhone'] == "") & (comp_cont['Company_Phone'] == ""))]
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].replace('', np.nan)
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].astype(float)
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].dt.tz_convert('Asia/Taipei')
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].dt.strftime('%Y-%m-%d %H:%M:%S')
comp_cont_mobile = comp_cont[comp_cont['MobilePhone'] != ""]
comp_cont_no_mobile = comp_cont[comp_cont['MobilePhone'] == ""]

comp_cont_mobile = comp_cont_mobile.sort_values(by='LastWorkDate', ascending=False)
comp_cont_mobile = comp_cont_mobile.drop_duplicates(subset=['MobilePhone'])
comp_cont_no_mobile = comp_cont_no_mobile.sort_values(by='LastWorkDate', ascending=False)
comp_cont_no_mobile = comp_cont_no_mobile.drop_duplicates(subset=['displayname', 'Company_Phone'])

tab = comp_cont_no_mobile['Company_Phone'].value_counts().reset_index()
tab.columns = ['Company_Phone', 'Freq']
comp_cont_no_mobile_nodup = comp_cont_no_mobile[comp_cont_no_mobile['Company_Phone'].isin(tab[tab['Freq'] < 2]['Company_Phone'])]
comp_cont_no_mobile_dup = comp_cont_no_mobile[~comp_cont_no_mobile['Company_Phone'].isin(tab[tab['Freq'] < 2]['Company_Phone'])]


comp_cont_final = pd.concat([comp_cont_mobile, comp_cont_no_mobile_nodup])

####################################主營城市###############################################

#pivot = comp_cont_final.groupby('DataRegion').size().reset_index(name='case_count')
comp_cont_final['城市'] = comp_cont_final['城市'].str.get(0)
comp_cont_final = comp_cont_final[comp_cont_final['城市']=='是']



['SG-INV' 'SG-A2' 'SG-A1' 'MY-A1' 'SG-N1' 'SG-A6' '印度分公司' 'US-East'
 'HK-INV' 'MY-A2' 'HK-E' 'HK-A' 'HK-D' 'HK-C' 'MY-INV' 'ID-A' 'ID-D'
 'ID-B' 'PH-A2' 'TH-A3' 'VN-A1' 'TH-A2' 'ID-C' 'TH-A1' 'PH-A1' 'PD'
 'TH-INV' 'VN-INV' 'MY-A3' 'PH-INV' 'ID-INV' '香港分公司' 'CAN' 'ATND(無效資料)'
 'US-West' 'PH-N1' 'MY-N1' 'TH-N1' 'DE' 'TH-Others' 'JP' 'US-Central' 'AU'
 'US-INV' 'KR' '台灣-海外' 'Intl Dept.' 'United States Branch' '資訊部' '數據中心課'
 'VN-N1' 'ID-N1' 'HK-N1' 'Thailand Branch' 'NZ' '績效組' 'PH-加盟' 'AU-加盟'
 'VN-加盟' 'Singapore Branch' '(開發)北區' 'Japan Branch' 'Philippines Branch'
 'AU-雪梨']
